# RAGwithContextFusion

### How to build a RAG System with Weaviate, BigQuery, and DSPy

Retrieval-Augmented Generation (RAG) systems combine the power of Large Language Models with knowledge sources, such as databases.

This tutorial will show you how to use DSPy to combine multiple knowledge sources, using Weaviate for vector search on chunks from the Weaviate blog post and Google's BigQuery for structured information about the authors of the blogs, such as their names, what team they work on at Weaviate, how many blogs they have written, and whether they are an active member of the Weaviate team.

We will use DSPy to create our RAGwithContextFusion agent to route queries, convert natural language queries into SQL commands to send to BigQuery, and use the acquired context to answer questions. DSPy uses the Gemini LLM under the hood.

![alt text](./bigquery-images/RAGwithContextFusion.png "Title Text")

# Connect DSPy to the Gemini API

![alt text](./bigquery-images/Gemini.png "Title Text")

Image source: https://gemini.google.com/

In [2]:
import dspy

gemini_pro = dspy.Google(model="gemini-pro", api_key=GOOGLE_API_KEY)

dspy.settings.configure(lm=gemini_pro)

gemini_pro("say hello")

/Users/cshorten/Desktop/DSPy-local/cohere_fix/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Hello!']

In [133]:
gemini_pro("What is Google BigQuery?")

["**Google BigQuery** is a fully managed, serverless data warehouse that enables fast and cost-effective analysis of large datasets. It is a cloud-based service that allows users to store, query, and analyze data at scale.\n\n**Key Features:**\n\n* **Massive Scalability:** BigQuery can handle datasets up to petabytes in size, making it suitable for large-scale data analysis.\n* **Fast Query Performance:** BigQuery uses a distributed processing engine to execute queries quickly, even on massive datasets.\n* **Serverless Architecture:** BigQuery is a fully managed service, eliminating the need for infrastructure management and maintenance.\n* **Cost-Effective:** BigQuery charges only for the data stored and the queries executed, making it a cost-effective solution for data analysis.\n* **Standard SQL Support:** BigQuery supports standard SQL, making it easy for users to write complex queries and perform advanced data analysis.\n* **Integration with Google Cloud Platform:** BigQuery seaml

# Load Unstructured Text Data into Weaviate

![alt text](./bigquery-images/weaviate-logo.png "Title Text")

# Load Unstructured Text Data into Memory

In [3]:
# read markdowns from disk
import os
import re

def chunk_list(lst, chunk_size):
    """Break a list into chunks of the specified size."""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

def split_into_sentences(text):
    """Split text into sentences using regular expressions."""
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

def read_and_chunk_index_files(main_folder_path):
    """Read index.md files from subfolders, split into sentences, and chunk every 5 sentences."""
    blog_chunks = []
    for folder_name in os.listdir(main_folder_path):
        subfolder_path = os.path.join(main_folder_path, folder_name)
        if os.path.isdir(subfolder_path):
            index_file_path = os.path.join(subfolder_path, 'index.mdx')
            if os.path.isfile(index_file_path):
                with open(index_file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    sentences = split_into_sentences(content)
                    sentence_chunks = chunk_list(sentences, 5)
                    sentence_chunks = [' '.join(chunk) for chunk in sentence_chunks]
                    blog_chunks.extend(sentence_chunks)
    return blog_chunks

# Example usage
main_folder_path = '../../../llm-frameworks/data'
blog_chunks = read_and_chunk_index_files(main_folder_path)
print(blog_chunks[0])

---
title: Combining LangChain and Weaviate
slug: combining-langchain-and-weaviate
authors: [erika]
date: 2023-02-21
tags: ['integrations']
image: ./img/hero.png
description: "LangChain is one of the most exciting new tools in AI. It helps overcome many limitations of LLMs, such as hallucination and limited input lengths."
---
![Combining LangChain and Weaviate](./img/hero.png)

Large Language Models (LLMs) have revolutionized the way we interact and communicate with computers. These machines can understand and generate human-like language on a massive scale. LLMs are a versatile tool that is seen in many applications like chatbots, content creation, and much more. Despite being a powerful tool, LLMs have the drawback of being too general.


# Create a Weaviate Schema and Import Data

In [289]:
import weaviate
import weaviate.classes.config as wvcc
from weaviate.util import get_valid_uuid
from uuid import uuid4

client = weaviate.connect_to_local()

weaviate_blog_chunks = client.collections.create(
    name = "WeaviateBlogChunk",
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_cohere(
        model="embed-english-v3.0"
    ),
    properties=[
        wvcc.Property(name="content", data_type=wvcc.DataType.TEXT)
    ]
)

for idx, blog_chunk in enumerate(blog_chunks):
    upload = weaviate_blog_chunks.data.insert(
        properties={
            "content": blog_chunk
        }
    )

# Query Test

In [290]:
response = weaviate_blog_chunks.query.hybrid(
    query="How does the Golang Garbage Collector work?",
    limit=1
)

for obj in response.objects:
    print(obj.properties)

{'content': "In a garbage-collected language, such as Go, C#, or Java, the programmer doesn't have to deallocate objects manually after using them. A GC cycle runs periodically to collect memory no longer needed and ensure it can be assigned again. Using a garbage-collected language is a trade-off between development complexity and execution time. Some CPU time has to be spent at runtime to run the GC cycles. Go's Garbage collector is highly concurrent and [quite efficient](https://tip.golang.org/doc/gc-guide#Understanding_costs)."}


# Load Structured Data into BigQuery

From cloud.google.com/bigquery, "BigQuery is a fully managed, AI-ready data analytics platform that helps you maximize value from your data and is designed to be multi-engine, multi-format, and multi-cloud". For example, companies often store information about transactions or customer relationships in structured tables.

![alt text](./bigquery-images/bigquery.png "Title Text")

Image source: https://cloud.google.com/bigquery

# Connect to BigQuery

Download the `google-cloud-bigquery` Python client with `pip`!

This tutorial is written with google-cloud-bigquery==3.21.0

In [65]:
!pip install google-cloud-bigquery > /dev/null


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [66]:
from google.cloud import bigquery
import google.auth

from google.oauth2 import service_account

# Replace with your Google Cloud credentials
credentials = service_account.Credentials.from_service_account_file(
    './google_auth.json')

In [67]:
bigquery_client = bigquery.Client(
    project="bigquery-playground-422417",
    credentials=credentials
)

## Google Cloud Data Marketplace with BigQuery

You can access many datasets in the Google Cloud Data Marketplace!

Maybe your RAG application needs to know what the most commonly occuring names of residents in Texas are!

In [336]:
QUERY = (
    'SELECT name, number FROM `bigquery-public-data.usa_names.usa_1910_2013` '
    'WHERE state = "TX" '
    'LIMIT 100')
query_job = bigquery_client.query(QUERY)
rows = query_job.result()

for idx, row in enumerate(rows):
    if idx > 2:
        break
    print(row)

Row(('Ruby', 314), {'name': 0, 'number': 1})
Row(('Louise', 127), {'name': 0, 'number': 1})
Row(('Carrie', 63), {'name': 0, 'number': 1})


Learn more about the [Google Cloud Marketplace in the 95th Weaviate Podcast](https://www.youtube.com/watch?v=UdAtsuoEd38) with **Dai Vu**, Director of Google Cloud Marketplace and ISV GTM and **Bob van Luijt**, Weaviate Co-Founder and CEO!

![alt text](./bigquery-images/gcp-pod.png "Title Text")

## Custom Schema

Schema created in the Google Cloud console.

In [69]:
table_ref = bigquery_client.dataset("WeaviateBlogs").table("BlogInfo")

print(bigquery_client.get_table("WeaviateBlogs.BlogInfo"))

# Define the schema fields
schema_fields = [
    bigquery.SchemaField("Name", "STRING"),
    bigquery.SchemaField("Team", "STRING"),
    bigquery.SchemaField("Blogs_Written", "INTEGER"),
    bigquery.SchemaField("Active_Weaviate_Team_Member", "BOOLEAN")
]

rows_to_insert = [
    ("Abdel Rodriguez", "Applied Research", 5, True),
    ("Adam Chan", "Developer Growth", 1, True),
    ("Ajit Mistry", "Developer Growth", 1, True),
    ("Alea Abed", "Marketing", 2, True),
    ("Amir Houieh", "Unbody", 1, False),
    ("Asdine El Hrychy", "Applied Research", 1, True),
    ("Bob van Luijt", "CEO Team", 5, True),
    ("Charles Frye", "Modal", 1, False),
    ("Connor Shorten", "Applied Research", 14, True),
    ("Dan Dascalescu", "Developer Relations", 6, True),
    ("Daniel Phiri", "Developer Relations", 3, True),
    ("Dave Cuthbert", "Developer Relations", 2, True),
    ("Dirk Kulawiak", "Core Engineering", 5, True),
    ("Edward Schmuhl", "Developer Growth", 2, True),
    ("Erika Cardenas", "Partnerships", 20, True),
    ("Etienne Dilocker", "CTO Team", 9, True),
    ("Femke Plantinga", "Developer Growth", 1, True),
    ("Ieva Urbaite", "Marketing", 2, True),
    ("Jerry Liu", "LlamaIndex", 1, False),
    ("John Trengrove", "Applied Research", 2, True),
    ("Jonathan Tuite", "Sales Engineering", 2, True),
    ("Joon-Pil (JP) Hwang", "Developer Relations", 18, True),
    ("Laura Ham", "Product", 7, False),
    ("Leonie Monigatti", "Developer Growth", 4, True),
    ("Marion Nehring", "Developer Relations", 1, True),
    ("Mohd Shukri Hasan", "Sales Engineering", 3, True),
    ("Peter Schramm", "Weaviate Cloud Services", 1, False),
    ("Sam Stoelinga", "Substratus AI", 1, False),
    ("Sebastian Witalec", "Developer Relations", 7, True),
    ("Stefan Bogdan", "Customer Success", 1, True),
    ("Tommy Smith", "Core Engineering", 3, True),
    ("Victoria Slocum", "Developer Growth", 1, True),
    ("Zain Hasan", "Developer Relations", 20, True)
]

errors = bigquery_client.insert_rows(
    table_ref,
    rows_to_insert,
    selected_fields=schema_fields
)

if errors == []:
    print("Rows inserted successfully.")
else:
    print("Errors occurred during insertion:")
    for error in errors:
        print(error)

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_8009/3365144275.py:1: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  table_ref = bigquery_client.dataset("WeaviateBlogs").table("BlogInfo")


bigquery-playground-422417.WeaviateBlogs.BlogInfo
Rows inserted successfully.


In [70]:
QUERY = (
    'SELECT * FROM bigquery-playground-422417.WeaviateBlogs.BlogInfo'
)

query_job = bigquery_client.query(QUERY)
rows = query_job.result()

for row in rows:
    print(row)
    break

Row(('Abdel Rodriguez', 'Applied Research', 5, True), {'Name': 0, 'Team': 1, 'Blogs_Written': 2, 'Active_Weaviate_Team_Member': 3})


# Storing Monitoring Logs in Weaviate *and* BigQuery

In [315]:
file_path = './WeaviateBlogRAG-0-0-0.json'

with open(file_path, 'r') as file:
    dataset = json.load(file)

# Get BigQuery Table
table_ref = bigquery_client.dataset("WeaviateBlogs").table("RAGLogs")

schema_fields = [
    bigquery.SchemaField("query", "STRING"),
    bigquery.SchemaField("answer", "STRING"),

]

# Create Weaviate Collection
rag_log_weaviate = client.collections.create(
    name = "RAGLog",
    # Embed with Cohere
    vectorizer_config=wvcc.Configure.Vectorizer.text2vec_cohere(
        model="embed-english-v3.0"
    ),
    properties=[
        wvcc.Property(name="query", data_type=wvcc.DataType.TEXT),
        wvcc.Property(name="answer", data_type=wvcc.DataType.TEXT)
    ]
)

# Import Data
for row in dataset:
    # Import to BigQuery
    rows_to_insert = [(row["query"], row["gold_answer"])]
    bigquery_client.insert_rows(
        table_ref,
        rows_to_insert,
        selected_fields=schema_fields
    )
    # Import to Weaviate
    upload = rag_log_weaviate.data.insert(
        properties={
            "query": row["query"],
            "answer": row["gold_answer"]
        }
    )

/var/folders/41/8dp_379x15d8zz4ppsjthdw40000gn/T/ipykernel_8009/497179841.py:7: PendingDeprecationWarning: Client.dataset is deprecated and will be removed in a future version. Use a string like 'my_project.my_dataset' or a cloud.google.bigquery.DatasetReference object, instead.
  table_ref = bigquery_client.dataset("WeaviateBlogs").table("RAGLogs")


# RAGwithContextFusion Program

![alt text](./bigquery-images/dspy.png "Title Text")

Image source: https://dspy-docs.vercel.app/

Now we will turn to our RAGwithContextFusion program that uses the:

- Blog chunks stored in Weaviate
- Author metadata stored in BigQuery
- RAG logs stored in Weaviate
- RAG logs stored in BigQuery

To answer questions, the program will route queries to the appropriate information sources, looping when multiple rounds of queries are needed.

# DSPy Signatures and Route Enum

In [316]:
from pydantic import BaseModel
from enum import Enum

class Route(Enum):
    Author_Info_BigQuery = "Author_Info_BigQuery"
    RAG_Log_BigQuery = "RAG_Log_BigQuery"
    RAG_Log_Weaviate = "RAG_Log_Weaviate"
    Blogs_Weaviate = "Blogs_Weaviate"

class TextToSQL(dspy.Signature):
    """Translate the natural language query into a valid SQL query for the given schema"""
    
    sql_schema_with_description: str = dspy.InputField()
    natural_language_query: str = dspy.InputField()
    sql_query: str = dspy.OutputField(desc="Only output the SQL query string without any newline characters.")
    
class QueryRouter(dspy.Signature):
    """Given a query and a list of data sources, output the best data source for answering the query."""
    
    query: str = dspy.InputField(desc="The query to answer with information from one of the data sources.")
    data_sources: str = dspy.InputField(desc="A description of each data source.")
    route: Route = dspy.OutputField()
        
class AgentLoopCondition(dspy.Signature):
    """Assess the context and search history and determine if enough context has been gathered to answer the question or if more context must be acquired from the information sources."""
    
    query: str = dspy.InputField(desc="The query to answer with information from the context.")
    data_sources: str = dspy.InputField(desc="A description of each data source.")
    contexts: str = dspy.InputField(desc="The context acquired so far.")
    more_info_needed: bool = dspy.OutputField(desc="Whether or not the question can be answered based on the context provided.")
    
class GenerateAnswer(dspy.Signature):
    """Asess the context and answer the question. 
Some context may be missing depending on the information sources the query router determined were needed to answer the question."""
    
    question: str = dspy.InputField()
    contexts: str = dspy.InputField(desc="Information acquired from searching multiple data sources.")
    data_sources: str = dspy.InputField(desc="A description of the data sources the contexts were acquired from.")
    answer: str = dspy.OutputField()

# Database Tools

In [317]:
from typing import List

class BigQuerySearcher():
    def __init__(self, sql_schema_with_description: str, 
                 bigquery_client: google.cloud.bigquery.client.Client):
        self.text_to_sql = dspy.TypedPredictor(TextToSQL)
        self.sql_schema_with_description = sql_schema_with_description
        self.bigquery_client = bigquery_client
    
    def sql_results_to_text(self,rows: bigquery.table.RowIterator) -> str:
        results = []
        for row in rows:
            row_strings = [f"{column}: {row[column]}" for column in row.keys()]
            result_string = ", ".join(row_strings)
            results.append(result_string)
    
        return "\n".join(results)
    
    def forward(self, query: str):
        sql_query = self.text_to_sql(natural_language_query=query, 
                            sql_schema_with_description=self.sql_schema_with_description).sql_query
        query_job = self.bigquery_client.query(sql_query)
        sql_results = query_job.result()
        text_sql_results = self.sql_results_to_text(sql_results)
        return text_sql_results
    
class WeaviateSearcher():
    def __init__(self, weaviate_client: weaviate.client.WeaviateClient,
                 collection_name: str,
                 view_properties: List[str]):
        self.collection = weaviate_client.collections.get(collection_name)
        self.view_properties = view_properties
    
    # ToDo, set `view_properties` as an Optional argument
    def parse_weaviate_response(self, response: weaviate.collections.classes.internal.QueryReturn):
        string_output = []
        for index, obj in enumerate(response.objects, start=1):
            result = {}
            for prop in self.view_properties:
                if prop in obj.properties:
                    result[prop] = obj.properties[prop]
            string_output.append(f"[{index}] {result}")
        return "\n".join(string_output)
    
    def forward(self, query: str):
        response = self.collection.query.hybrid(
            query=query,
            limit=3
        )
        return self.parse_weaviate_response(response)

# Structured Schema Info and Data Source Metadata

In [318]:
author_schema_with_description = """
Technical Schema Information:

Table: bigquery-playground-422417.WeaviateBlogs.BlogInfo
Attributes: 
`Name` STRING
`Team` STRING
`Blogs_Written` INTEGER
`Active_Weaviate_Team_Member` BOOLEAN

Description of the Table:

The table contains information about Weaviate Blog post authors.
The `Name` attribute is the name of the author.
The `Team` attribute is the particular team the author works on at Weaviate.
The `Blogs_Written` attribute is the number of blogs the author has written.
The `Active_Weaviate_Team_Member` attribute denotes whether the author is currently a member of the Weaviate team.
"""

rag_log_schema_with_description = """
Technical Schema Information:

Table: bigquery-playground-422417.WeaviateBlogs.RAGLog
Attributes:
`query` STRING
`answer` STRING

Description of the Table:

The table contains questions submitted to a question answering system and the resulting response from the system.
The `query` attribute is the query sent to the system.
The `answer` attribute is the system's response to the query.
"""

route_config = {
    "data_sources": {
        "Author_Info_BigQuery": BigQuerySearcher(author_schema_with_description, bigquery_client),
        "RAG_Log_BigQuery": BigQuerySearcher(rag_log_schema_with_description, bigquery_client),
        "RAG_Log_Weaviate": WeaviateSearcher(weaviate_client, "RAGLog", ["query", "answer"]),
        "Blogs_Weaviate": WeaviateSearcher(weaviate_client, "WeaviateBlogChunk", ["content"])
    },
    "description": """
        Author_Info_BigQuery: Structured SQL table in BigQuery that contains information about authors of Weaviate blog posts such as their `Name`, the `Team` they work on at Weaviate, the number of `Blogs_Written` from the author, and whether they are an `Active_Weaviate_Team_Member`.
        RAG_Log_BigQuery: Structured SQL table in BigQuery that contains questions submitted to a question answering system and the system's response.
        RAG_Log_Weaviate: A Vector Index that contains questions submitted to a question answering system and the system's response.
        Blogs_Weaviate: A Vector Index that contains snippets from Weaviate's blog posts."""
}

# RAGwithContextFusion

In [333]:
class RAGwithContextFusion(dspy.Module):
    def __init__(self, route_config):
        self.route_config = route_config
        self.query_router = dspy.TypedPredictor(QueryRouter)
        self.agent_loop_condition = dspy.TypedPredictor(AgentLoopCondition)
        self.generate_answer = dspy.TypedPredictor(GenerateAnswer)
    
    def forward(self, query):
        enough_context = False
        contexts, queries = [], []
        while not enough_context:
            query_route = self.query_router(query=query, data_sources=self.route_config["description"]).route.name
            context = self.route_config["data_sources"][query_route].forward(query=query)
            contexts.append(context)
            queries.append(query)
            query_history = "\n".join(f"query {i+1}: {query}" for i, query in enumerate(queries))
            contexts_str = "\n".join(f"context {i+1}: {item}" for i, item in enumerate(contexts))
            enough_context = self.agent_loop_condition(query=query,
                                                       data_sources=self.route_config["description"],
                                                       contexts=contexts_str).more_info_needed
        answer = self.generate_answer(question=query,
                                      contexts=contexts_str, data_sources=self.route_config["description"]).answer
        return dspy.Prediction(answer=answer)

In [324]:
rag_with_context_fusion = RAGwithContextFusion(route_config)

In [332]:
rag_with_context_fusion(query="Who are the most frequent authors of Weaviate blog posts?")

Prediction(
    answer='Zain Hasan and Erika Cardenas are the most frequent authors of Weaviate blog posts, with 20 posts each.'
)

In [326]:
rag_with_context_fusion(query="How does ref2vec work?")

Prediction(
    answer='Ref2Vec infers a centroid vector from a user\'s references to other vectors. This vector is updated in real-time to reflect the user\'s preferences and actions. Ref2Vec integrates with Weaviate through the "user-as-query" method, where the user\'s vector is used as a query to fetch relevant products.'
)